# MIDIProcessor
The `MIDIProcessor` class is designed to extract musical features from a given MIDI file. These features include:
* Key (limited to major and minor scales)
* MIDI type (e.g., composite, chord, bass, melody, arp, drums — composite indicates the MIDI may contain both chord progressions and melodies)
* Duration (in Bar-Step-Tick metrical time)
* Stems (chord, bass, melody)
* Chord progression
* Groove

MIDI files are parsed using the `pretty-midi` library. The note sequence is then converted into a 2D array representation, which is essentially a piano roll. We use the bar-step-tick concept in TOMI to adopt metrical time instead of absolute
time. In the current implementation, the start and end positions of each MIDI note are quantized to the nearest step instead of tick to reduce the size of the piano roll.

All feature extraction algorithms are rule-based and operate on the piano roll instead of the raw note sequence.

To aid in visualization, we also provide a textual version of the piano roll for use in terminal environments:

<div><img src="../../../img/midi_processor.jpg" style="width:600px" alt=""></div>

In [2]:
from tomi import MIDIProcessor, RollPrintingMode
import warnings
warnings.filterwarnings('ignore')

In [3]:
# create a MIDIProcessor objects
midi1_composite = MIDIProcessor('./midi1_composite.mid')

# if you want to load the MIDI into a different BPM, try:
# midi1_composite = MIDIProcessor('./midi1_composite.mid', target_bpm=100) # The MIDI will be converted to BPM 100.

### Baisc Features

In [4]:
def print_features(midi: MIDIProcessor):
    print(f"File: {midi.name}\nBPM: {midi.bpm}\nKey: {midi.key}\nDuration (Seconds): {midi.midi.end_time}\nDuration (BarStepTick): {midi.midi.end_bst}\nCeil BST: {midi.ceil_bst}")

print_features(midi1_composite)
# "3b15s23t(4/4)" means 3 bars 15 steps and 23 ticks in 4/4 time signature.

File: ./midi1_composite.mid
BPM: 120
Key: CMajor
Duration (Seconds): 7.994791666666666
Duration (BarStepTick): 3b15s23t(4/4)
Ceil BST: 4b(4/4)


### Piano Roll Visualization
There are 3 view modes:
* RollPrintingMode.Compact (only displays the pitch rows containing notes)
* RollPrintingMode.Normal (only displays the pitch rows from the top note row to the bottom note row)
* RollPrintingMode.Full (displays all 128 pitch rows)

In the view, the "▆" blocks represent note durations, and the taller "█" blocks indicate note onsets. A dimmer color indicates a lower note velocity.

In [4]:
# visualize MIDI piano roll and meta-info
# Compact mode (default)
midi1_composite.piano_roll.print_roll(mode=RollPrintingMode.Compact)

View: Compact, Type: Composite, MIDI: ./midi1_composite.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


In [5]:
# Normal mode
midi1_composite.piano_roll.print_roll(mode=RollPrintingMode.Normal)

View: Normal, Type: Composite, MIDI: ./midi1_composite.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


In [6]:
# Full mode
midi1_composite.piano_roll.print_roll(mode=RollPrintingMode.Full)

View: Full, Type: Composite, MIDI: ./midi1_composite.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


### Scale Shadow
We can add the scale shadow into the piano roll to check which notes are in the scale and which are not. Currently we only support major and minor scales.

In [7]:
midi1_composite.piano_roll.print_roll(mode=RollPrintingMode.Normal, scale_shadow=True)

View: Normal, Type: Composite, MIDI: ./midi1_composite.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


### Stem Extraction
We can extract bass, chord, and melody stems from the MIDI file.

Note that the algorithm has limitations. **The extracted bass and chord stems retain only the fundamental pitches and do not preserve the original rhythmic patterns.** For example, if a chord is played with a repeating rhythm like "X X X X " (e.g., four 16th-note strikes), it may be interpreted as a single sustained chord "X-------". Note that when processing complex MIDI files such as broken chords, the algorithm may also misclassify chords/bass notes.

In [8]:
midi1_composite.bass_roll.print_roll()
midi1_composite.chord_roll.print_roll()
midi1_composite.melody_roll.print_roll()

View: Compact, Type: Bass, MIDI: ./midi1_composite.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


View: Compact, Type: Chord, MIDI: ./midi1_composite.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


View: Compact, Type: Melody, MIDI: ./midi1_composite.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


In [9]:
# A groovy midi case
midi2_groovy = MIDIProcessor('./midi2_groovy.mid')
midi2_groovy.piano_roll.print_roll()

View: Compact, Type: Composite, MIDI: ./midi2_groovy.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


In [10]:
midi2_groovy.bass_roll.print_roll()
midi2_groovy.chord_roll.print_roll()
midi2_groovy.melody_roll.print_roll()

View: Compact, Type: Bass, MIDI: ./midi2_groovy.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


View: Compact, Type: Chord, MIDI: ./midi2_groovy.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


View: Compact, Type: Melody, MIDI: ./midi2_groovy.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


In [11]:
# another case, with messy notes
midi3 = MIDIProcessor('./midi3.mid')
midi3.piano_roll.print_roll()

View: Compact, Type: Composite, MIDI: ./midi3.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


In [12]:
midi3.bass_roll.print_roll()
midi3.chord_roll.print_roll()
midi3.melody_roll.print_roll()

View: Compact, Type: Bass, MIDI: ./midi3.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


View: Compact, Type: Chord, MIDI: ./midi3.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


View: Compact, Type: Melody, MIDI: ./midi3.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor


In [15]:
midi7_comp = MIDIProcessor('./midi7_comp.mid')
midi7_comp.piano_roll.print_roll()

View: Compact, Type: Composite, MIDI: ./midi7_comp.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: D#Major/CMinor


In [14]:
midi7_comp.bass_roll.print_roll()
midi7_comp.chord_roll.print_roll()
midi7_comp.melody_roll.print_roll()

View: Compact, Type: Bass, MIDI: ./midi7_comp.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: D#Major/CMinor


View: Compact, Type: Chord, MIDI: ./midi7_comp.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: D#Major/CMinor


View: Compact, Type: Melody, MIDI: ./midi7_comp.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: D#Major/CMinor


# More MIDI Types
Now let’s look at the different MIDI types. Currently, there are nine supported types: Composite, Chord, Bass, Melody, Arp, Kick, ClapSnare (backbeat), HiHat, and Drummer. The `MIDIProcessor` can automatically detect the type based on keywords found in the MIDI file name. If no explicit type is specified in the name, it will attempt to infer whether the file is a Bass, Melody, or Drummer based on the piano roll. If none of these are detected, the MIDI is classified as Composite by default.

In [6]:
# Arpeggio
midi4_arp = MIDIProcessor('./midi4_arp.mid')
midi4_arp.piano_roll.print_roll()

View: Compact, Type: Arp, MIDI: ./midi4_arp.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 8bars 0steps 0ticks 
key: DMajor/BMinor


In [7]:
# Drum
midi5_drum = MIDIProcessor('./midi5_drum.mid')
midi5_drum.piano_roll.print_roll()

View: Compact, Type: Drummer, MIDI: ./midi5_drum.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 2bars 0steps 0ticks 
key: Unknown


In [8]:
# Bass
midi6_bass = MIDIProcessor('./midi6_bass.mid')
midi6_bass.piano_roll.print_roll()

View: Compact, Type: Bass, MIDI: ./midi6_bass.mid, TimeSignature: 4/4
Quantization Mode: Start_end, Quantization Factor: 16, Length: 4bars 0steps 0ticks 
key: CMajor/AMinor
